## Data cleaning

In [1]:
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from sklearn import svm
from nltk.stem import WordNetLemmatizer
from nltk.tag.stanford import StanfordPOSTagger
from nltk.parse.stanford import StanfordDependencyParser

with open('dara 1_train.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    """
    for row in reader:
        print(row['example_id'])
        print(row[' text'])
        print(row[' aspect_term'])
        print(row[' term_location'])
        print(row[' class'])
        print()        
    """
    happy_emoji = [':‑)',':)',':-]',':]',':-3',':3',':->',':>','8-)','8)',':-}', ':}',':o)',':c)',':^)','=]','=)',
                  ':‑D',':D','8‑D','8D','x‑D','xD','X‑D','XD','=D','=3','B^D',':-))']
                
    sad_emoji = [':‑(',':(',':‑c',':c',':‑<',':<',':‑[',':[',':-||','>:[',':{',':@','>:(']
    
    negative_words = ['doesn’t', 'isn’t', 'wasn’t', 'shouldn’t', 'wouldn’t', 'couldn’t', 'won’t', 'can’t', 'don’t']
    
    tokenizer = RegexpTokenizer(r'\w+')
    
    stop_words = set(stopwords.words('english'))
    
    singles = []
    texts = []
    tempText = ''
    classes = []
    aspectTerm = []
    
    for row in reader:
        aspectTerm.append(tokenizer.tokenize(row[' aspect_term']))
        
        classes.append(row[' class'])
        tempText = row[' text'].replace('[comma]',',').lower()
               
        for hem in happy_emoji:
            if hem in tempText:
                tempText = tempText.replace(hem, 'emojihappy')
        
        for sem in sad_emoji:
            if sem in tempText:
                tempText = tempText.replace(sem, 'emojisad')
                
        for negw in negative_words:
            if negw in tempText:
                tempText = tempText.replace(negw, 'not')
        
        singles.append(tokenizer.tokenize(tempText))
        
        texts.append(tempText)
        
    newTokenList = []
    
    '''
    # stemming
    stemmer = SnowballStemmer("english")
        
    for tokenList in singles:
        newToken = []
        for token in tokenList:
            if token not in stop_words:
                newToken.append(stemmer.stem(token))
            if token == "not":
                newToken.append(stemmer.stem(token))
        newTokenList.append(newToken)
    '''
    
    # lemmatize
    wnl = WordNetLemmatizer()
    
    for tokenList in singles:
        newToken = []
        for token in tokenList:
            if token not in stop_words:
                newToken.append(wnl.lemmatize(token))
        newTokenList.append(newToken)
    
    # lemmatize aspect term
    newApectTermList =[]
    
    for asp in aspectTerm:
        newAT = []
        for ap in asp:
            newAT.append(wnl.lemmatize(ap))
        newApectTermList.append(newAT)
        
    '''
    unigrams = []
    
    for single in singles:
        unigrams.append(ngrams(single, 1))
    '''
    # new review text
    newText = []
    
    for tk in newTokenList:
        newText.append(' '.join(tk))
        
    # new aspect term
    newAspectTerm = []
    
    for apt in newApectTermList:
        newAspectTerm.append(' '.join(apt))
        
    '''
    kf = KFold(n_splits=10)
    
    
    #vectorizer = TfidfVectorizer(use_idf=True)
    count_vectorizer = CountVectorizer()
    data_tfidf = count_vectorizer.fit_transform(newText)
    tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data_tfidf)
    '''
    
    '''
    # POS tagging
    st = StanfordPOSTagger('english-left3words-distsim.tagger')
    
    textPOS = []
    
    for nt in newText:
        textPOS.append(st.tag(nt.split()))
        
    # save pos tags to txt file
    f1 = open('postagsfull.txt','w')
    for row in textPOS:
        f1.write(str(row)+"\n")
    f1.close()
    '''
    
    
    # dependency parsing

    depParsingList = []
    
    
    for nt in newText[1900:2000]:
        dep_parser=StanfordDependencyParser(model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
        result = dep_parser.raw_parse(nt)    
        dep = result.__next__()    
        depParsingList.append(list(dep.triples()))
        
    
    f2 = open('dependencyparsingfull7.txt','w')
    for dp in depParsingList:
        f2.write(str(dp)+"\n")
    f2.close()
    
    
    # POS tagging
    
    

C:\Users\sanku\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
#experiment
#print(textPOS[0])
import sys
sys.getsizeof(newText)

In [ ]:
#experiment
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
print(wnl.lemmatize('features'))

In [ ]:
#data_train,data_test,target_train,target_test = cross_validation.train_test_split(tfidf_data,classes,test_size=0.4,random_state=43)
#classifier = BernoulliNB().fit(data_train,target_train)
#predicted = classifier.predict(data_test)
#evaluate_model(target_test,predicted)
from sklearn import metrics
from sklearn.metrics import classification_report
clf = svm.SVC(kernel='linear', C=1, random_state=0)
predicted = cross_val_predict(clf, tfidf_data, classes, cv=10)



print(classification_report(classes,predicted))
print("The accuracy score is {:.2%}".format(metrics.accuracy_score(classes, predicted)))

In [ ]:
#experiment
#from nltk.tag.stanford import CoreNLPPOSTagger
#CoreNLPPOSTagger(url='http://localhost:9000').tag('What is the airspeed of an unladen swallow ?'.split())

#import nltk
#nltk.pos_tag(newTokenList[0])
import csv
from nltk.tag.stanford import StanfordPOSTagger

st = StanfordPOSTagger('english-left3words-distsim.tagger')
newT = ["obviously one important feature computer human interface", 
         "good every day computing web browsing",
         "keyboard alright plate around cheap plastic make hollow sound using mouse command button"]
taggedT = []

for n in newT:
    taggedT.append(st.tag(n.split()))

'''
for tt in taggedT:
    print(tt)
'''

'''
out = open('out.csv', 'w')
for row in taggedT:
    out.write('%s;' % column)
out.write('\n')
out.close()

'''
'''
with open('out.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter = ']')
    for row in taggedT:        
        spamwriter.writerow(str(row))
'''
'''
f = open('out.csv','w')
for row in taggedT:        
    f.write(str(row) + "\n") 
f.close()
'''
f = open('postags.txt','w')
for row in taggedT:
    f.write(str(row)+"\n")
f.close()

dep_parser=StanfordDependencyParser(model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")

depParsingList = []
    
for nt in newT:
    result = dep_parser.raw_parse(nt)    
    dep = result.__next__()    
    depParsingList.append(list(dep.triples()))
    
for dp in depParsingList:
    print(dp)
    
f2 = open('dependencyparsing.txt','w')
for dp in depParsingList:
    f2.write(str(dp)+"\n")
f.close()

    

In [ ]:
#experiment
from nltk.parse.stanford import StanfordDependencyParser
dep_parser=StanfordDependencyParser(model_path="edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
print([parse.tree() for parse in dep_parser.raw_parse("The quick brown fox jumps over the lazy dog.")])
result = dep_parser.raw_parse("The pizza at the restaurant was very good.")
dep = result.__next__()
list(dep.triples())

In [ ]:
#experiment
import os
os.environ['STANFORD_MODELS'] + 

In [ ]:
#experiment
#from collections import Counter
'''
for text in texts:
    print('\n'+text)
'''
'''
for sss in singles:
    print(sss)
'''
'''
for tk in newTokenList:
    print(' '.join(tk))
'''
#print(newTokenList[0])
'''
for tk in newTokenList:
    print("\n",tk)
'''

'''
#print(Counter(unigrams))

for grams in unigrams:
    #print(Counter(grams))
    for g in grams:
         print(g)
'''

#len(classes)

for nt in newText:
    print("\n",nt)


'''
for tokenList in singles:
        for token in tokenList:
            print(token)
        print("\n")
'''

'''
for at in newAspectTerm:
    print("\n",at)
'''
'''
for dp in depParsingList:
    print(dp)
'''

In [ ]:
# big experiment

import csv

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import accuracy_score

# review.csv contains two columns
# first column is the review content (quoted)
# second column is the assigned sentiment (positive or negative)
def load_file():
    with open('review.csv') as csv_file:
        reader = csv.reader(csv_file,delimiter=",",quotechar='"')
        next(reader)
        data =[]
        target = []
        for row in reader:
            # skip missing data
            if row[0] and row[1]:
                data.append(row[0])
                target.append(row[1])

        return data,target

# preprocess creates the term frequency matrix for the review data set
def preprocess():
    data,target = load_file()
    count_vectorizer = CountVectorizer(binary='true')
    data = count_vectorizer.fit_transform(data)
    tfidf_data = TfidfTransformer(use_idf=False).fit_transform(data)

    return tfidf_data

def learn_model(data,target):
    # preparing data for split validation. 60% training, 40% test
    data_train,data_test,target_train,target_test = cross_validation.train_test_split(data,target,test_size=0.4,random_state=43)
    classifier = BernoulliNB().fit(data_train,target_train)
    predicted = classifier.predict(data_test)
    evaluate_model(target_test,predicted)

# read more about model evaluation metrics here
# http://scikit-learn.org/stable/modules/model_evaluation.html
def evaluate_model(target_true,target_predicted):
    print(classification_report(target_true,target_predicted))
    print("The accuracy score is {:.2%}".format(accuracy_score(target_true,target_predicted)))

def main():
    data,target = load_file()
    tf_idf = preprocess()
    learn_model(tf_idf,target)


main()


In [ ]:
#experiment

with open('dara 1_train.csv', newline='') as csvfile2:
    reader2 = csv.DictReader(csvfile2)
    
    data2,target2 = load_file()
                count_vectorizer = CountVectorizer(binary='true')
                data = count_vectorizer.fit_transform(data)
                tfidf_data = TfidfTransformer(use_idf=False).
                fit_transform(data)

                return tfidf_data

In [ ]:
#experiment
exps = ['asd','asd_','_wqeqwe']
expsf = []
for ex in exps:
    expsf.append(ex.replace('_',''))
print(expsf)

In [ ]:
#experiment
lhap = [':)',':P',':D']
lsad = [':(',':|']
t1 = "i am happy :), but he is sad :|"
t2 = "i am sad :(, he is :P"
t1.replace(lhap,'smilehappy')
t1.replace(lsad,'smilesad')
t2.replace(lhap,'smilehappy')
t2.replace(lsad,'smilesad')
print(t1)
print(t2)

In [ ]:
#experiment

happy_emoji = [':‑)',':)',':-]',':]',':-3',':3',':->',':>','8-)','8)',':-}', ':}',':o)',':c)',':^)','=]','=)',
                  ':‑D',':D','8‑D','8D','x‑D','xD','X‑D','XD','=D','=3','B^D',':-))']
                
sad_emoji = [':‑(',':(',':‑c',':c',':‑<',':<',':‑[',':[',':-||','>:[',':{',':@','>:(']

strr = "i am happy :)"
for hem in happy_emoji:
    if hem in strr:
        strr = strr.replace(hem, 'emojihappy')

for sem in sad_emoji:
    strr.replace(sem, 'emojisad')
    
print(strr)
